# Iterations metrics
The OPF results are stored in a csv file that can be read using the pandas function pandas.read_csv.
use the function reXplan.config.path.engineDatabaseFile(simulationName) to get the results file path.

In [ ]:
import reXplan as rx
import pandas as pd

rx.config.path.SetWorkspaceFolder('Examples')
simulationName = '1-HV-urban--0-sw'

df = pd.read_csv(rx.config.path.engineDatabaseFile(simulationName), index_col = [0, 1, 2, 3, 4]) # read database with results
df = rx.utils.filter_non_converged_iterations(df) # filterining non-converged iterations

Iterations dropped: [21, 27]


Using plotly it is possible to visulize the number of lines in service for each iteration.

In [2]:
import plotly.express as px

df_line = rx.utils.group_by(df, 'sum', 'iteration', 'field', 'type').loc[:,:,'line']
px.line(rx.utils.invert(df_line), x=rx.utils.invert(df_line).index, y = 'in_service', color = 'iteration')

Instead of ploting all iterations, it is possible to only plot specific quantiles

In [3]:
df_line_quantiles = rx.utils.invert(rx.utils.get_quantiles_on_iterations(df_line, [0.05,0.5,0.95]))
px.line(df_line_quantiles, x=df_line_quantiles.index, y = 'in_service', color = 'quantile')

Similarly it is possible to plot the loss of load for the selected quantiles.

In [5]:
df_load = rx.utils.group_by(rx.utils.filter(df, type = 'load'), 'sum', 'iteration', 'field', 'type')
df_load_quantiles = rx.utils.invert(rx.utils.get_quantiles_on_iterations(df_load, [0.05, 0.25, 0.5, 0.75, 0.95]))
df_load_quantiles['loss_of_load_p_percentage'] = (df_load_quantiles['loss_of_load_p_mw'])/df_load_quantiles['max_p_mw'] *100
px.line(df_load_quantiles, x=df_load_quantiles.index, y = 'loss_of_load_p_percentage', color = 'quantile')

In the following plot the distribution of the energy not served is shown for each timestamp

In [6]:
df_network = rx.utils.invert(rx.utils.filter(df, type = 'network')) # filter network fields and invert for plotting
px.scatter(df_network, x=df_network.index, y= 'energy_not_served_mwh' )

# Montercalo metrics
The energy not served probability distribution

In [7]:
df_network_condensed = rx.utils.filter(df, type = 'network').sum(axis = 1) # sum over timesteps
df_network_condensed_ = rx.utils.invert(df_network_condensed)
px.histogram(df_network_condensed_, x='energy_not_served_mwh', histnorm='probability')

EENS (Estimated Energy Not Served) and LOLE (Loss Of Load)

In [8]:
statistics= df_network_condensed.groupby('field').mean() # average over iterations
EENS = statistics['energy_not_served_mwh']
LOLE = statistics['loss_of_load_p_duration_h']
print(f'EENS : {EENS.round(2)} MWh, LOLE : {LOLE.round(2)} h')

EENS : 754.04 MWh, LOLE : 22.96 h


Survivability: Probability of supplying at minimum percentage of the load.

In [9]:
crt_loss_of_load = 20
df_loss_of_load = df.loc[:,:,"loss_of_load_p_percentage","network"]
Survivability = pd.DataFrame(1 - (df_loss_of_load > crt_loss_of_load).sum() / df_loss_of_load.index.levels[1].size, columns = [simulationName])
px.line(Survivability*100).update_layout(xaxis_title="Time", yaxis_title="Survivability [%]")